In [1]:
from itertools import chain
import sys

sys.path.append('../')

In [2]:
from ml_metrics import apk
import numpy as np
import torch
from torch import nn

In [3]:
from iftorch.algorithms import SVDRecommender, UserItemEmbeddingsFeature
from iftorch.dataset import UserItemSet
from iftorch.loss import ModifiedMaxWarpLoss

## Load Train/Test UserItemSet

In [4]:
train_set = UserItemSet.load_cls_from_file('trainset', '../data')
test_set = UserItemSet.load_cls_from_file('testset', '../data')

In [5]:
svd_recommender = SVDRecommender.load_cls_from_file('svd_recommender_test', '../data')

In [6]:
user_item_embeddings = UserItemEmbeddingsFeature(
    trainable=True,
    user_embeddings=svd_recommender.user_embeddings_,
    item_embeddings=svd_recommender.item_embeddings_.transpose()
)

In [7]:
class CrossProduct(nn.Module):
    def __init__(self, is_cuda=True):
        super(CrossProduct, self).__init__()
        if is_cuda:
            self._torch = torch.cuda
        else:
            self._torch = torch
    
    def forward(self, inputs):
        user_embeddings = inputs[0]
        item_embeddings = inputs[1]

        return torch.sum(user_embeddings * item_embeddings, dim=1, keepdim=True)

cross_product_model = CrossProduct()

In [8]:
optimizer = torch.optim.Adam(
    chain(user_item_embeddings.user_embeddings.parameters(),
          user_item_embeddings.item_embeddings.parameters()), 
    lr=1e-5, 
    eps=1e-8
)

In [9]:
modified_max_warp_loss = ModifiedMaxWarpLoss(
    num_epochs=100,
    batch_size=30,
    verbose=10.0
)

In [10]:
def scoring_method(self, users):
    """Scoring method after training UserItemEmbeddingsFeature.
    
    Parameters
    ----------
    users : np.ndarray[int], [len(users), 1]
    
    Return
    ------
    np.ndarra[float], [len(users) x self._features.n_items,]
    """
    user_embeddings = (
        self._features
        .user_embeddings(self._torch.LongTensor(users))
        .cpu().detach().numpy()
    )

    item_embeddings = (
        self._features.item_embeddings(
            self._torch.LongTensor(np.arange(self._features.n_items))
        ).cpu().detach().numpy()
    )

    return np.reshape(user_embeddings.dot(item_embeddings.transpose()), -1)

In [11]:
modified_max_warp_loss.fit(train_set, cross_product_model, optimizer, user_item_embeddings, scoring_method)

Epoch : 0 | Loss : 3.7300e+00
Epoch : 1 | Loss : 3.7292e+00
Epoch : 2 | Loss : 3.7301e+00
Epoch : 3 | Loss : 3.7160e+00
Epoch : 4 | Loss : 3.7091e+00
Epoch : 5 | Loss : 3.7020e+00
Epoch : 6 | Loss : 3.6868e+00
Epoch : 7 | Loss : 3.6900e+00
Epoch : 8 | Loss : 3.6763e+00
Epoch : 9 | Loss : 3.6631e+00
Epoch : 10 | Loss : 3.6664e+00
Epoch : 11 | Loss : 3.6617e+00
Epoch : 12 | Loss : 3.6398e+00
Epoch : 13 | Loss : 3.6366e+00
Epoch : 14 | Loss : 3.6278e+00
Epoch : 15 | Loss : 3.6268e+00
Epoch : 16 | Loss : 3.6242e+00
Epoch : 17 | Loss : 3.6131e+00
Epoch : 18 | Loss : 3.6023e+00
Epoch : 19 | Loss : 3.5900e+00
Epoch : 20 | Loss : 3.5813e+00
Epoch : 21 | Loss : 3.5666e+00
Epoch : 22 | Loss : 3.5664e+00
Epoch : 23 | Loss : 3.5698e+00
Epoch : 24 | Loss : 3.5431e+00
Epoch : 25 | Loss : 3.5372e+00
Epoch : 26 | Loss : 3.5255e+00
Epoch : 27 | Loss : 3.5068e+00
Epoch : 28 | Loss : 3.5042e+00
Epoch : 29 | Loss : 3.5105e+00
Epoch : 30 | Loss : 3.4942e+00
Epoch : 31 | Loss : 3.5021e+00
Epoch : 32 | Loss 

ModifiedMaxWarpLoss(batch_size=30, gap=1.0, is_cuda=True, max_abs_score=100.0,
          num_epochs=100, num_negative_samples=100, verbose=10.0)

In [12]:
user_item_embeddings.save_to_file(
    'trained_user_item_embeddings',
    '../data'
)

In [13]:
user_item_embeddings = UserItemEmbeddingsFeature.load_cls_from_file(
    'trained_user_item_embeddings', '../data'
)

modified_max_warp_loss = ModifiedMaxWarpLoss(
    num_epochs=0,
    batch_size=30,
    verbose=10.0
)

modified_max_warp_loss.fit(train_set,
                           cross_product_model,
                           optimizer,
                           user_item_embeddings,
                           scoring_method)

ModifiedMaxWarpLoss(batch_size=30, gap=1.0, is_cuda=True, max_abs_score=100.0,
          num_epochs=0, num_negative_samples=100, verbose=10.0)

In [14]:
user2recommended_items = modified_max_warp_loss.predict(
    test_set, 
    train_set, 
    remove_users_not_in_train=True,
    max_score_to_filter=0.0
)

In [15]:
mapk_scores = []
for user, recommended_items in user2recommended_items.items():
    mapk_scores.append(apk(test_set.user2items_inner[user],
                           recommended_items,
                           k=100))

In [16]:
np.mean(mapk_scores)

0.0007349616576189105